In [ ]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torchlensmaker as tlm

from xxchallenge import *


# Parameters
A = tlm.parameter(-1.62)
T = 0.0
NY = tlm.parameter(90.0)
NZ = tlm.parameter(0.0)
RZ = 0

cylinder = tlm.ImplicitCylinder(*torch.tensor([-50/2, 50/2, 37.02/2], dtype=torch.float64).unbind())

# Primary mirror
primary = tlm.Parabola(1800, A=A, normalize=True)
primary_data = StoreVar(lambda data: data)

# Optical model
optics = tlm.Sequential(
    tlm.Gap(-1000),
    XXLightSource.load(),
    #RaysViewerPlane(2000, "input"),
    tlm.Gap(998),   ## bottom margin
    tlm.Translate3D(y=T),
    tlm.Rotate3D(z=RZ),
    primary_data,
    tlm.ReflectiveSurface(primary),
    Focus(primary),
    #tlm.FocalPoint(),
    BoxLoss(450, 1.0, 0.1),
    tlm.Rotate3D(y=NY, z=NZ),
    NonImagingRod(cylinder),
)

xxrender(optics, sampling={"xx": 50, "letter": "both"})

In [ ]:
         

param_groups = [
    {'params': [A], 'lr': 1e-4},
]

record = tlm.optimize(
    optics,
    optimizer = optim.SGD(param_groups),
    sampling = {"xx": 100, "disable_viewer": True, "letter": "positive"},
    dim = 3,
    num_iter = 100
)


plot_record(record, param_groups, optics)
record.best()

settobest(optics, record, param_groups)

print("A", A)
F = 1./(primary._sag.unnorm(primary.diameter / 2) * 4)
print("F", F.item())

xxrender(optics, sampling={"xx": 50, "letter": "both"})